<a href="https://colab.research.google.com/github/osnavarrom/IS_Project/blob/main/Proyecto_Sistemas_inteligentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install dataprep --q
!pip install deap --q
!pip install scikit-fuzzy --q
!pip install linearmodels --upgrade --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from deap import base, creator, tools, algorithms
#import skfuzzy as fuzz
import statsmodels.api as sm
import linearmodels


In [7]:
!file -bi /content/PIB_Deptos_01.csv

text/csv; charset=iso-8859-1


In [8]:
# Leer el archivo CSV
#df = pd.read_csv('/content/PIB_Deptos.csv', encoding='iso-8859-1')
df = pd.read_csv('/content/PIB_Deptos_01.csv', encoding='iso-8859-1')

# Vista previa del DataFrame con los datos
df.head(10)

,Codigo_Depto,Tag,Departamento,1980,1981,1982,1983,1984,1985,1986,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,COL,COLOMBIA,2072.290883,2601.331893,3289.212871,4032.181395,5074.932739,6528.791485,8846.879269,...,714093.000000,762903.000000,804692.000000,863782.000000,920471.000000,987791.000000,1.060068e+06,998471.000000,1.192634e+06,1.469791e+06
1,5,Ant,Antioquia,343.324302,438.461257,536.423283,640.452767,840.834472,1078.243766,1520.120823,...,98572.848540,106818.961300,115446.257000,126021.644100,132368.590700,141680.080300,1.535594e+05,148361.905400,1.778665e+05,2.124525e+05
2,8,Atl,Atlántico,86.578191,107.806082,138.885439,182.144869,221.837905,274.211406,363.326296,...,29112.427030,32382.665870,35716.295690,38575.333100,40874.780230,43368.866170,4.657475e+04,44832.642840,5.277107e+04,6.369644e+04
3,11,Bog,Bogotá D.C.,494.001954,643.315986,818.609045,1001.580821,1257.409821,1579.936245,2063.996069,...,177360.955600,191025.601700,206478.408900,221455.570300,236785.897500,253941.024500,2.735244e+05,260747.484400,3.030944e+05,3.616981e+05
4,13,Bol,Bolívar,55.816771,65.211968,78.970489,106.525052,134.193402,175.693307,232.559712,...,25858.703060,26623.462390,28105.211290,30917.858850,33393.780520,35410.286210,3.828530e+04,34625.951320,4.225557e+04,5.164319e+04
5,15,Boy,Boyacá,61.887369,79.472017,94.865581,114.914980,134.590687,170.905781,221.489372,...,19380.059110,20839.474100,22164.683370,23671.130610,24782.262570,26884.207240,2.857355e+04,26480.063660,3.118401e+04,3.923349e+04
6,17,Cal,Caldas,42.820732,52.788022,66.407179,87.111169,106.437724,135.272925,191.146898,...,10497.853680,11518.624890,12513.501740,13798.295280,14749.498120,15710.828930,1.701244e+04,16674.101380,1.957675e+04,2.390982e+04
7,18,Caq,Caquetá,8.927432,11.308912,14.287726,18.975185,24.026851,30.016622,42.552670,...,2835.584119,3175.189159,3349.867477,3665.700439,3866.274572,4071.614961,4.313453e+03,4170.644755,4.778224e+03,5.569984e+03
8,19,Cau,Cauca,35.011237,40.115331,52.688402,67.294811,81.422033,101.247442,140.197645,...,11624.412630,13085.761670,14622.036080,16070.029530,16738.902200,17478.959530,1.875536e+04,18193.667870,2.138001e+04,2.611343e+04
9,20,Ces,Cesar,31.150656,39.514708,46.557375,56.543241,69.861676,96.043193,129.731160,...,13383.024190,13653.577180,14570.402490,16992.102500,19550.611970,21262.810490,2.072453e+04,17710.496390,2.330254e+04,3.828247e+04


In [9]:
# Reestructurar el DataFrame con melt
df_melted = df.melt(id_vars=['Codigo_Depto', 'Tag', 'Departamento'],
                    var_name='Year',
                    value_name='PIB')

# Convertir la columna 'Year' a tipo numérico
df_melted['Year'] = df_melted['Year'].astype(int)

# Mostrar el DataFrame final
df_melted

,Codigo_Depto,Tag,Departamento,Year,PIB
0,0,COL,COLOMBIA,1980,2072.290883
1,5,Ant,Antioquia,1980,343.324302
2,8,Atl,Atlántico,1980,86.578191
3,11,Bog,Bogotá D.C.,1980,494.001954
4,13,Bol,Bolívar,1980,55.816771
...,...,...,...,...,...
1457,95,Guv,Guaviare,2022,3199.895760
1458,86,Put,Putumayo,2022,3541.160563
1459,88,San,San Andrés,2022,8421.254880
1460,97,Vau,Vaupés,2022,2306.537925


In [10]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Codigo_Depto  1462 non-null   int64  
 1   Tag           1462 non-null   object 
 2   Departamento  1462 non-null   object 
 3   Year          1462 non-null   int64  
 4   PIB           1462 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 57.2+ KB


In [11]:
df_melted.describe()

,Codigo_Depto,Year,PIB
count,1462.000000,1462.00000,1.462000e+03
mean,50.617647,2001.00000,2.185237e+04
std,31.599167,12.41392,9.496762e+04
min,0.000000,1980.00000,5.051085e-01
25%,19.000000,1990.00000,3.129751e+02
50%,51.000000,2001.00000,2.270351e+03
75%,81.000000,2012.00000,9.781502e+03
max,99.000000,2022.00000,1.469791e+06


In [12]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Codificación One-Hot de la columna 'Tag'
encoder = OneHotEncoder(sparse=False)
encoded_deptos = encoder.fit_transform(df_melted[['Tag']])

# Crear un DataFrame con las variables categóricas codificadas
encoded_df = pd.DataFrame(encoded_deptos, columns=encoder.get_feature_names_out(['Tag']))

# Concatenar las columnas codificadas con el DataFrame original
df_encoded = pd.concat([df_melted, encoded_df], axis=1).drop('Tag', axis=1)

# Escalar la columna 'GDP'
scaler = MinMaxScaler()
df_encoded['PIB'] = scaler.fit_transform(df_encoded[['PIB']])

# Eliminar las columnas redundantes
df_encoded.drop(['Codigo_Depto', 'Departamento','Tag_COL'], axis=1, inplace=True)

# Mostrar el DataFrame preparado
df_encoded



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Year,PIB,Tag_Ama,Tag_Ant,Tag_Ara,Tag_Atl,Tag_Bog,Tag_Bol,Tag_Boy,Tag_Cal,...,Tag_Put,Tag_Qui,Tag_Ris,Tag_San,Tag_Sat,Tag_Suc,Tag_Tol,Tag_Val,Tag_Vau,Tag_Vic
0,1980,0.001410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1980,0.000233,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1980,0.000059,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1980,0.000336,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1980,0.000038,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2022,0.002177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,2022,0.002409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1459,2022,0.005729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1460,2022,0.001569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
#df_encoded.info()
df_encoded

,Year,PIB,Tag_Ama,Tag_Ant,Tag_Ara,Tag_Atl,Tag_Bog,Tag_Bol,Tag_Boy,Tag_Cal,...,Tag_Put,Tag_Qui,Tag_Ris,Tag_San,Tag_Sat,Tag_Suc,Tag_Tol,Tag_Val,Tag_Vau,Tag_Vic
0,1980,0.001410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1980,0.000233,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1980,0.000059,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1980,0.000336,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1980,0.000038,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2022,0.002177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,2022,0.002409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1459,2022,0.005729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1460,2022,0.001569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Extract features and target variable
features = list(df_scaled.columns[:-1])
target = df_scaled.columns[-1]

X = df_scaled.select_dtypes(np.number).columns
y = df_scaled['Tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [43, 34]

In [23]:
X

,Tag,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,COL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,Ant,0.165470,0.168341,0.162848,0.158606,0.165486,0.164954,0.171601,0.166189,0.172982,...,0.137164,0.136680,0.138778,0.142495,0.144887,0.142665,0.142259,0.143753,0.147608,0.147925
2,Atl,0.041545,0.041199,0.041953,0.044912,0.043486,0.041773,0.040809,0.039067,0.040251,...,0.038176,0.039256,0.041067,0.043301,0.043531,0.043134,0.042597,0.042700,0.043800,0.042886
3,Bog,0.238199,0.247111,0.248664,0.248192,0.247590,0.241816,0.233095,0.238682,0.238179,...,0.245695,0.247187,0.249313,0.255750,0.255501,0.256255,0.256063,0.257067,0.260295,0.253076
4,Bol,0.026698,0.024821,0.023732,0.026153,0.026211,0.026680,0.026024,0.027809,0.025960,...,0.033117,0.034692,0.033507,0.033832,0.034656,0.034996,0.034529,0.034871,0.033566,0.034056
5,Boy,0.029628,0.030304,0.028566,0.028234,0.026290,0.025946,0.024772,0.025158,0.025074,...,0.025958,0.025605,0.025915,0.026442,0.026256,0.025628,0.025886,0.025697,0.025399,0.024760
6,Cal,0.020425,0.020044,0.019912,0.021337,0.020741,0.020487,0.021341,0.021177,0.019877,...,0.013086,0.013147,0.013680,0.014434,0.014813,0.014714,0.014559,0.014777,0.015566,0.015014
7,Caq,0.004065,0.004094,0.004062,0.004434,0.004498,0.004362,0.004541,0.004579,0.004646,...,0.002276,0.002400,0.002727,0.003034,0.003069,0.002875,0.002759,0.002782,0.003029,0.002588
8,Cau,0.016655,0.015171,0.015740,0.016421,0.015810,0.015275,0.015581,0.015003,0.015129,...,0.014025,0.014727,0.015737,0.017058,0.017446,0.016878,0.016351,0.016424,0.017090,0.016528
9,Ces,0.014792,0.014940,0.013875,0.013754,0.013532,0.014477,0.014397,0.014344,0.014250,...,0.019438,0.017194,0.016482,0.016993,0.018515,0.019937,0.020187,0.018284,0.016605,0.018142
